### **Model Details**

Convolutional neural network model that runs on RGB images and predicts human joint locations of a single person. The model is designed to be run in the browser using Tensorflow.js or on devices using TF Lite in real-time, targeting movement/fitness activities. This variant: MoveNet.SinglePose.Thunder is a higher capacity model (compared to MoveNet.SinglePose.Lightning) that performs better prediction quality while still achieving real-time (<30FPS) speed. Naturally, thunder will lag behind the lightning, but it will pack a punch.

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/Presentation-MoveNet.gif?raw=true" width=500></center>

#### **Model Architecture**

MobileNetV2 image feature extractor with Feature Pyramid Network decoder (to stride of 4) followed by CenterNet prediction heads with custom post-processing logics. Lightning uses depth multiplier 1.75.

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/ArchiMoveNet.png?raw=true" width=700></center>

https://ieeexplore.ieee.org/document/9406043  
  
  
https://www.kaggle.com/models/google/movenet/frameworks/tensorFlow2/variations/multipose-lightning  
https://www.tensorflow.org/hub/tutorials/movenet  
https://www.tensorflow.org/lite/tutorials/pose_classification


**Inputs**

A frame of video or an image, represented as an int32 tensor of dynamic shape: 1xHxWx3, where H and W need to be a **multiple of 32** and the larger dimension is recommended to be 256. To prepare the input image tensor, one should resize (and pad if needed) the image such that the above conditions are hold. Please see the Usage section for more detailed explanation. Note that the size of the input image controls the tradeoff between speed vs. accuracy so choose the value that best suits your application. The channel order is RGB with values in [0, 255].

**Outputs**

A float32 tensor of shape [1, 6, 56].

* The first dimension is the batch dimension, which is always equal to 1.
* The second dimension corresponds to the maximum number of instance detections. The model can detect up to **6** people in the image frame simultaneously.
* The third dimension represents the predicted bounding box/keypoint locations and scores.
  * The first **17 * 3** elements are the keypoint locations and scores in the format: [y_0, x_0, s_0, y_1, x_1, s_1, …, y_16, x_16, s_16], where y_i, x_i, s_i are the yx-coordinates (normalized to image frame, e.g. range in [0.0, 1.0]) and confidence scores of the i-th joint correspondingly.
  * The order of the 17 keypoint joints is: [nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle].
  * The remaining 5 elements [ymin, xmin, ymax, xmax, score] represent the region of the bounding box (in normalized coordinates) and the confidence score of the instance.

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/keypoints.png?raw=true" width=500></center>

**Usage**

The following code snippet shows how to load and run the model inference on an input image in Python. We recommend to resize and pad your image to the height/width such that:

* The height/width are both multiple of 32.
* The height to width ratio is close (and enough) to cover the original image's aspect ratio.
* Make the larger side to be 256 (one should adjust this based on the speed/accuracy requirements). For example, a 720p image (i.e. 720x1280 (HxW)) should be resized and padded to 160x256 image.

### Téléchargement du modèle

In [ ]:
!wget --no-check-certificate -O MultiPosteLightning.tar.gz 'https://docs.google.com/uc?export=download&id=1jVkGdxpEBOFyvgmr7q9R-xCUzw2WGRGo'
!mkdir SavedModel
!tar -xf MultiPosteLightning.tar.gz --directory SavedModel
!rm MultiPosteLightning.tar.gz

# Utilisation du modèle

### Chargement du modèle

In [ ]:
import tensorflow as tf

model_path = "SavedModel"
model = tf.saved_model.load(model_path)

### Chargement d'une image dans le tenseur d'entrée

In [ ]:
from PIL import Image
import numpy as np
from io import BytesIO

img_data = tf.io.gfile.GFile('models/research/object_detection/test_images/image2.jpg', 'rb').read()
image = Image.open(BytesIO(img_data))
(im_width, im_height) = image.size
image_np  = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
input_tensor = np.expand_dims(image_np, 0)